# FM FT API: Data Validation and \$Token Estimation

#### Usage Scenario:
This notebook goes hand-in-hand with Databricks-Mosaicml's FT API. Our customers may find it useful in scenarios where there is a risk of data being malformed. It acts as a preventive measure to ensure data integrity and helps in cost assessment for the fine-tuning process.

#### Script Purpose:
- **Not for Training**: This script is not utilized during the training process.
- **Ad-Hoc Validation**: It serves as an ad-hoc utility for users to run independently prior to starting fine-tuning.
- **Data Verification**: Its primary function is to validate the user's data before they invoke the Fine-Tuning (FT) API.
- **Cost Estimation**: Users can estimate the cost implications with this script.

#### Note on Long-Term Solution:
- **Temporary Measure**: This script is a stop-gap solution.
- **Future Development**: We are in the process of developing a long-term data preparation service, which will eventually replace this script.

#### User Defines:
- The inputs to this validation script is assumed to be the same or a subset of the FT API arguments, i.e., a configuration like below. Is this a valid assumption?
- For the reference, FT API expects following
```
cfg = {
    model: str,
    train_data_path: str,
    save_folder: str,
    *,
    task_type: Optional[str] = "INSTRUCTION_FINETUNE",
    eval_data_path: Optional[str] = None,
    eval_prompts: Optional[List[str]] = None,
    custom_weights_path: Optional[str] = None,
    training_duration: Optional[str] = None,
    learning_rate: Optional[float] = None,
    context_length: Optional[int] = None,
    experiment_trackers: Optional[List[Dict]] = None,
    disable_credentials_check: Optional[bool] = None,
    timeout: Optional[float] = 10,
    future: Literal[False] = False,
}
``` 

# Installation

In [0]:
%pip uninstall -y llm-foundry

In [0]:
dbutils.library.restartPython()

In [0]:
# %pip install git+https://github.com/mosaicml/llm-foundry.git@byod/data_validation
%pip install --upgrade git+https://github.com/XiaohanZhangCMU/llm-foundryX.git@validation 

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import re
import json
import tempfile
import numpy as np
import pandas as pd 
from collections import defaultdict
from argparse import ArgumentParser, Namespace

import datasets 

from llmfoundry.utils import (create_om_cfg, token_counts_and_validation, token_counts, 
        check_HF_datasets, is_hf_dataset_path, is_uc_delta_table,
        pandas_processing_fn, integrity_check, convert_text_to_mds,
        _args_str, plot_hist, dataframe_to_mds)

import transformers
transformers.logging.set_verbosity_error()

# Instruction Fine Tuning

#### User Defines

In [0]:
FT_API_args = Namespace(
    model='EleutherAI/gpt-neox-20b',
    train_data_path= 'main.streaming.random_large_table', # '/Volumes/main/mosaic_hackathon/managed-volume/IFT/train.jsonl', # 'tatsu-lab/alpaca/train', # , # 'tatsu-lab/alpaca/train',  # 'mosaicml/dolly_hhrlhf/train', # tatsu-lab/alpaca/train',
    task_type='INSTRUCTION_FINETUNE',
    training_duration=3,
    context_length=2048,
)

temporary_jsonl_data_path = '/Volumes/main/mosaic_hackathon/managed-volume/IFT/ft_data_11Jan24_3/train'
# os.environ['HF_ASSETS_CACHE'] = '/tmp/'
# os.environ['HF_HOME'] = '/tmp/'
# os.environ['HF_HUB_CACHE'] = '/tmp/'
os.environ['HF_DATASETS_CACHE'] = '/tmp/'
os.makedirs(temporary_jsonl_data_path, exist_ok=True)

#### Data Loading

The IFT data needs to stay with a format 
```
prompt: xxx
response or completion: yyy
```

Based on FT_API_args.train_data_path, we will select an ingestion method from three options.

- Option-1. Your data is a JSONL file which stores in an object store supported by Composer. [Example file to-be-added](todo - add a link to such a file)
- Option-2. You provide a Huggingface dataset ID. Note you need to provide a split as well. [Example dataset link to-be-added](huggingface.co)
- Option-3. You have a delta table. 

In [0]:
raw_dataset = None

if is_hf_dataset_path(FT_API_args.train_data_path):
    check_HF_datasets(FT_API_args.train_data_path)
    dataset_id, split = '/'.join(FT_API_args.train_data_path.split('/')[:2]), FT_API_args.train_data_path.split('/')[-1]    
    raw_dataset = datasets.load_dataset(dataset_id, split=split)       
else:
    if is_uc_delta_table(FT_API_args.train_data_path):    
        df = spark.read.table(FT_API_args.train_data_path).toPandas()
        df.to_json(os.path.join(temporary_jsonl_data_path, 'data.jsonl'), orient='records', lines=True)
        raw_dataset = datasets.Dataset.from_pandas(df) 
        FT_API_args.train_data_path = temporary_jsonl_data_path
    else: 
        # train_data_path is a jonsl file (local/remote)
        from composer.utils import dist, get_file, parse_uri 
        data_path = FT_API_args.train_data_path 
        backend, _, _ = parse_uri(data_path)
        if backend not in ['', None]: # It's a remote path, download before loading it
            with tempfile.TemporaryDirectory() as tmp_dir:
                destination = os.path.join(tmp_dir, 'data.jsonl')
                get_file(data_path, destination)
                df = pd.read_json(destination, orient='records', lines=True)    
        else: 
            df = pd.read_json(data_path, orient='records', lines=True)    

        raw_dataset = datasets.Dataset.from_pandas(df)
        FT_API_args.train_data_path = os.path.dirname(data_path)

if raw_dataset is None: 
    raise RuntimeError("Can't find a proper ingestion method")

#### Validation

In [0]:
# Initial dataset stats
print("Num examples:", len(raw_dataset))
print("First example:")
for ex in raw_dataset: 
    print(ex)
    print() 
    break 

_ALLOWED_RESPONSE_KEYS = {'response', 'completion'}
_ALLOWED_PROMPT_KEYS = {'prompt'}
format_errors = defaultdict(int)

for ex in raw_dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1 
        continue 
    
    found = False 
    for key in _ALLOWED_PROMPT_KEYS:
        prompts = ex.get(key, None)
        if prompts:
            found = True 
    if not found: 
        format_errors["missing_prompt"] += 1

    found = False
    for key in _ALLOWED_RESPONSE_KEYS:        
        responses = ex.get("response", None)
        if responses: 
            found = True 
    if not found:
        format_errors["missing_response"] += 1
        
if format_errors:
    print("Oops! Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("Congratulations! No errors found")    

#### Token Estimation

Tokenize the raw dataset and we see some statistics of the tokens and estimate the overall cost based on default trainining duration

In [0]:
n_epochs = FT_API_args.training_duration if FT_API_args.training_duration is not None else 1 
batch_tokens = token_counts(FT_API_args)
n_billing_tokens_in_dataset = sum(batch_tokens['ntokens'])

In [0]:
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, ~{n_epochs * n_billing_tokens_in_dataset} tokens will be used in training")
plot_hist(pd.Series(batch_tokens['ntokens']))

In [0]:
# all_tokens = token_counts_and_validation(FT_API_args)
# plot_hist(pd.Series(all_tokens))
# pd.Series(all_tokens).max(), max(batch_tokens['ntokens'])

# Continued Pretrain

#### User Defines

In [0]:
FT_API_args = Namespace(
    model='EleutherAI/gpt-neox-20b',
    train_data_path= '/Volumes/main/mosaic_hackathon/managed-volume/ABT',
    task_type='CONTINUED_PRETRAIN',
    training_duration=3,
    context_length=2048,
)
temporary_mds_output_path = '/Volumes/main/mosaic_hackathon/managed-volume/mds_data_11Jan24_5'
# temporary_mds_output_path = '/tmp/CPT/mds_data_11Jan24_4'

In [0]:
!rm -rf {temporary_mds_output_path}

#### Ingestion, Tokenization and Materialization

CPT takes a folder of txt files as input. It tokenize the text fields and materialize as a streaming dataset of MDS format. 

FT API uses [llmfoundry/scripts/data_prep/convert_text_to_mds.py](https://github.com/mosaicml/llm-foundry/blob/main/scripts/data_prep/convert_text_to_mds.py) to download all the txt files and convert them to MDS. 

In this notebook, we provide two additional approaches via Spark and Dask. 

**Warning** CPT datasets are normally much larger than IFT, so the tokenization and materialization can be very time consuming. 

**1. Delta Ingestion --> Spark Dataframe:** 

If you don't have a single-user-assigned cluster and DBR < 14.3, move on to option-2. 

Otherwise, you can leverage Delta Ingestion's tools to ingest the folder of txt files as a Spark dataframe and have the schema automatically inferred. 

In [0]:
# dbutils.fs.ls(FT_API_args.train_data_path)
output_location = FT_API_args.train_data_path + '/*.txt'
df = spark.sql("SELECT * FROM read_files('%s')" % output_location).withColumnRenamed('value', 'text')
df.show(2)

**2. Dask.bag --> Dask.DataFrame:**  

If you are on UC enabled clusters where mapInPandas does not work, you can try Dask. 

Dask uses the current node as a ```Local Cluster```

In [0]:
import dask.bag as db

input_folder = FT_API_args.train_data_path
pattern = input_folder + '/*.txt'
b = db.read_text(pattern, linedelimiter='\n', blocksize='128MiB')
df = b.to_dataframe(columns = ['text'])
df = df[df.text != '\n']

**3. dataframe_to_mds + tokenization:**  

dataframe_to_mds is a utility function. It takes either a dask dataframe or a Spark dataframe, and a tokenization function and convert raw txt to MDS dataset. 

In [0]:
mds_kwargs = {
    'out': temporary_mds_output_path,
    'columns': {
        'tokens': 'bytes'
    },
    'keep_local': True, 
}
udf_kwargs = {
    'concat_tokens': FT_API_args.context_length,
    'tokenizer': FT_API_args.model, 
    'eos_text': '',
    'compression': 'zstd',
    'no_wrap': False,
    'bos_text': '',
}
dataframe_to_mds(df,
                 merge_index=True,
                 mds_kwargs=mds_kwargs,
                 udf_iterable=pandas_processing_fn,
                 udf_kwargs=udf_kwargs)

#### Validation

We perform integrity checks on MDS dataset
- number of shards match with index.json. 
- Inspect first 5 examples by decode the tokens back to texts

In [0]:
print("Num examples:", len(df))
print("First example:")
for ex in df['text']: 
    print(ex)
    print() 
    break 

if not integrity_check(temporary_mds_output_path): 
    raise ValueError("MDS has not been created correctly. There are missing shards!")

In [0]:
# Sanity Check
import numpy as np
from streaming import StreamingDataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(FT_API_args.model)
tokenizer.model_max_length = 5000000000  # Hack to prevent warnings from HuggingFace
mds_dataset = StreamingDataset(local=temporary_mds_output_path, shuffle=False)
for i in range(5):
    l = np.frombuffer(mds_dataset[i]['tokens'], dtype=np.int64)
    print(''.join(tokenizer.decode(l)))
    print()

#### Token Estimation

In [0]:
MAX_TOKENS_PER_EXAMPLE = FT_API_args.context_length if FT_API_args.context_length is not None else 4096
TARGET_EPOCHS = FT_API_args.training_duration if FT_API_args.training_duration is not None else 1 
n_epochs = TARGET_EPOCHS

n_billing_tokens_in_dataset = len(mds_dataset) * FT_API_args.context_length 
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, ~{n_epochs * n_billing_tokens_in_dataset} tokens will be used in training")